In [1]:
import os
import re
import os
import re
import Crypto.Cipher.AES
import Crypto.Util.Padding
import base64
import json

In [2]:
def process_py_file(file_path):
    with open(file_path, "r", encoding="utf8") as f:
        file_content = f.read()

    # Remove import statements
    file_content = re.sub(r"^import.*$|^from .* import.*$", "", file_content, flags=re.MULTILINE)

    file_content = re.sub(r"^sys.path.append.*\n", "", file_content, flags=re.MULTILINE)

    # Remove comments
    file_content = re.sub(r"#.*", "", file_content)

    # # Remove function descriptions and docstrings
    patron_docstrings_triple_comillas = r'(\"\"\")(.*?)(\"\"\")'
    patron_docstrings_comillas_simples = r'(\'\'\')(.*?)(\'\'\')'
    patron_total = f"{patron_docstrings_triple_comillas}|{patron_docstrings_comillas_simples}"

    file_content = re.sub(patron_total, '', file_content, flags=re.DOTALL)

    # Remove the code inside if __name__ == "__main__": block
    file_content = re.sub(r'if __name__ == "__main__":[\s\S]*', "", file_content)

    # Remove any empty lines and extra whitespaces
    file_content = re.sub(r"^\s*\n", "", file_content, flags=re.MULTILINE)
    file_content = re.sub(r"\n\s*\n", "\n", file_content)
    file_content = file_content.strip()

    return file_content


In [3]:
def get_import_lines(file_path):
    with open(file_path, "r", encoding="utf8") as f:
        file_content = f.read()

    # Find all import lines using regular expression
    import_lines = re.findall(r"^import.*$|^from .* import.*$", file_content, re.MULTILINE)

    return import_lines



In [4]:
input_file_path = "../../discrete/distributions/bernoulli.py"
processed_content = process_py_file(input_file_path)
print(processed_content)

class BERNOULLI:
    def __init__(self, measurements):
        self.parameters = self.get_parameters(measurements)
        self.p = self.parameters["p"]
    def cdf(self, x: float) -> float:
        result = scipy.stats.bernoulli.cdf(x, self.p)
        return result
    def pmf(self, x: int) -> float:
        result = (self.p**x) * (1 - self.p) ** (1 - x)
        return result
    def get_num_parameters(self) -> int:
        return len(self.parameters)
    def parameter_restrictions(self) -> bool:
        v1 = self.p > 0 and self.p < 1
        return v1
    def get_parameters(self, measurements) -> dict[str, float | int]:
        p = measurements.mean
        parameters = {"p": p}
        return parameters


In [5]:
input_file_path = "../../discrete/distributions/bernoulli.py"
import_lines = get_import_lines(input_file_path)

In [6]:
IMPORTS = []
for file in os.listdir("../../discrete/distributions"):
    if ".py" in file:
        import_lines = get_import_lines(f"../../discrete/distributions/{file}")
        IMPORTS.extend(import_lines)

input_file_path = "../../discrete/measurements/measurements_discrete.py"
import_lines = get_import_lines(input_file_path)
IMPORTS.extend(import_lines)

In [7]:
CODE = "\n".join(sorted(list(set(IMPORTS)))) + "\n\n"
for file in os.listdir("../../discrete/distributions"):
    if ".py" in file:
        processed_content = process_py_file(f"../../discrete/distributions/{file}")
        CODE += processed_content + "\n\n"

In [8]:
input_file_path = "../../discrete/measurements/measurements_discrete.py"
measuerements_code = process_py_file(input_file_path)
CODE += measuerements_code + "\n\n"

input_file_path = "../../discrete/test_chi_square_discrete.py"
test_chi_square_discrete_code = process_py_file(input_file_path)
CODE += test_chi_square_discrete_code + "\n\n"

input_file_path = "../../discrete/test_kolmogorov_smirnov_discrete.py"
test_kolmogorov_smirnov_discrete_code = process_py_file(input_file_path)
CODE += test_kolmogorov_smirnov_discrete_code + "\n\n"

In [9]:
class_phitter_discrete_code = process_py_file("../../phitter/discrete/phitter_discrete.py")
CODE += class_phitter_discrete_code + "\n\n"

In [10]:
def encrypt_aes(texto: str, clave: str) -> str:
    data = texto.encode()
    key = clave.encode()

    cipher = Crypto.Cipher.AES.new(key, Crypto.Cipher.AES.MODE_CBC)
    ct_bytes = cipher.encrypt(Crypto.Util.Padding.pad(data, Crypto.Cipher.AES.block_size))
    iv = base64.b64encode(cipher.iv).decode("utf-8")
    ct = base64.b64encode(ct_bytes).decode("utf-8")

    result = json.dumps({"iv": iv, "ciphertext": ct})
    return result

In [11]:
clave = "X7g!2#4vKmQpR3sT5uW8yZbDfHjL9nV1" 

In [12]:
code_file = open("./production_discrete.py", "+w", encoding="utf8")
code_file.write(encrypt_aes(CODE, clave))
code_file.close()

In [13]:
if_name_code = """
if __name__ == "__main__":
    path = "../../discrete/data/data_binomial.txt"
    sample_distribution_file = open(path, "r", encoding="utf-8-sig")
    data = [float(x.replace(",", ".")) for x in sample_distribution_file.read().splitlines()]

    phitter_discrete = PHITTER_DISCRETE(data, confidence_level=0.95, minimum_sse=100)
    sorted_results_sse, not_rejected_results = phitter_discrete.fit()

    for distribution, results in not_rejected_results.items():
        print(f"Distribution: {distribution}, SSE: {results['sse']}, Aprobados: {results['n_test_passed']}")
"""


In [14]:
CODE += if_name_code

In [15]:
code_file = open("./test_production_discrete.py", "+w", encoding="utf8")
code_file.write(CODE)
code_file.close()